<a href="https://colab.research.google.com/github/vedantdave77/Natural_Laguage_Processing/blob/master/Random_forest(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Machine Learning Classifiers: Building a basic Random Forest model

### Read in & clean text

In [2]:

import nltk
nltk.download('stopwords')
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_tfidf_feat.head()


count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])

X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
X_count_feat.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,body_len,punct%,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,8064,8065,8066,8067,8068,8069,8070,8071,8072,8073,8074,8075,8076,8077,8078,8079,8080,8081,8082,8083,8084,8085,8086,8087,8088,8089,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto

### Explore RandomForestClassifier through Cross-Validation

In [0]:
from sklearn.model_selection import KFold, cross_val_score

In [6]:
rf = RandomForestClassifier(n_jobs= -1)
k_fold = KFold(n_splits = 5)
cross_val_score(rf, X_features,data['label'], cv = k_fold,scoring= 'accuracy', n_jobs= -1)


NameError: ignored

### Random Forest classifier from Holdout

In [0]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

NameError: ignored

In [9]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

NameError: ignored

In [0]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [0]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

### Grid Search
It is the method to train the algorithm with various number of hyper parameters, here we use the depth and estimator to get idea about the change.

In [0]:
def train_RF(n_est,depth):
  rf = RandomForestClassifier(n_estimators = n_est, max_depth =depth, n_jobs= -1) # -1 give parallel tree modeling running,
  rf_model = rf.fit(X_train, y_train)                                              # ... that save time 
  y_pred = rf.predict(X_test)
  score(y_test, y_pred)
  precision, recall, fscore, support = score(y_test ,y_pred ,pos_label ='spam',average = 'binary')
  print('Est: {} / Depth: {} ------Precision: {} / Recall: {} / Accuracy: {}'.format(n_est, depth,
    round(precision,3),round(recall,3),(y_pred == y_test).sum() / len(y_pred),3))



In [11]:
for n_est in [10, 50,100]:
  for depth in [10,20,30, 40]:
    train_RF(n_est,depth)

NameError: ignored

Observation:

Here, with increase of depth the recall value must increase and the precision value remain stable . onthe other side Accuracy is increase. shows the depth as true decider for the average of decision tree classification performance.

### Explorintg the parameters setting using Grid SearchCV

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV  

In [17]:
%%time
rf = RandomForestClassifier()
param = {'n_estimators': [10, 300],
         'max_depth': [30, None]}

gs = GridSearchCV(rf,param,cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat,data['label'])
ans = pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending= False)[0:5]

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 28.7 s, sys: 414 ms, total: 29.1 s
Wall time: 2min 56s


In [18]:
ans

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,32.711775,5.077258,0.284454,0.042041,None,300,"{'max_depth': None, 'n_estimators': 300}",0.974888,0.975741,0.975741,0.967655,0.972147,0.973235,0.003086,1
2,1.589039,0.059506,0.055283,0.001955,None,10,"{'max_depth': None, 'n_estimators': 10}",0.974888,0.969452,0.967655,0.958670,0.966757,0.967487,0.005234,2
1,15.266514,0.167783,0.186396,0.006142,30,300,"{'max_depth': 30, 'n_estimators': 300}",0.970404,0.959569,0.961366,0.954178,0.955975,0.960302,0.005658,3
0,1.009801,0.128270,0.059500,0.005343,30,10,"{'max_depth': 30, 'n_estimators': 10}",0.961435,0.958670,0.962264,0.949686,0.955975,0.957607,0.004536,4


In [19]:
%%time
rf = RandomForestClassifier()
param = {'n_estimators': [10, 300],
         'max_depth': [30, None]}

gs = GridSearchCV(rf,param,cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat,data['label'])
Ans = pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending= False)[0:5]

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 28.6 s, sys: 520 ms, total: 29.1 s
Wall time: 2min 55s


In [20]:
Ans

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,32.492899,5.108828,0.289822,0.048122,None,300,"{'max_depth': None, 'n_estimators': 300}",0.979372,0.975741,0.973944,0.967655,0.968553,0.973056,0.004413,1
2,1.773418,0.103253,0.066076,0.002157,None,10,"{'max_depth': None, 'n_estimators': 10}",0.971300,0.962264,0.968553,0.964061,0.961366,0.965511,0.003811,2
1,15.464072,0.361207,0.194455,0.005699,30,300,"{'max_depth': 30, 'n_estimators': 300}",0.968610,0.958670,0.963163,0.953279,0.952381,0.959224,0.006102,3
0,1.289902,0.205233,0.074059,0.013638,30,10,"{'max_depth': 30, 'n_estimators': 10}",0.960538,0.957772,0.959569,0.950584,0.960467,0.957787,0.003736,4
